In [ ]:
import os, sys
import pandas as pd
from pycaret.regression import *
sys.path.append(r"C:\work\water_use\CAWSC_WaterUse\WUtrainer")
from featurize import MultiOneHotEncoder
target = 'wu_rate'

# Load Dataset
## x

In [ ]:
dataset = pd.read_csv(r"C:\work\water_use\ml_experiments\annual_v_0_0\clean_train_db.csv")
pop_info = pd.read_csv(r"pop_info.csv")

In [ ]:
#pop_info
pop_info['pop'] = pop_info['pop_swud16'].copy()
mask = pop_info['pop'].isna() | pop_info['pop']==0
pop_info.loc[mask, 'pop'] = pop_info[mask]['plc_pop_interpolated']
mask = pop_info['pop'].isna() | pop_info['pop']==0
pop_info.loc[mask, 'pop'] = pop_info[mask]['TPOPSRV']
mask = pop_info['pop'].isna() | pop_info['pop']==0
pop_info.loc[mask, 'pop'] = pop_info[mask]['tract_pop']

In [ ]:
#pop_info['pop'] = pop_info['pop_swud16'].copy()

In [ ]:
#mask = pop_info['pop'].isna()
#pop_info.loc[mask, 'pop'] = pop_info[mask]['plc_pop_interpolated']

In [ ]:
#mask = pop_info['pop'].isna()
#pop_info.loc[mask, 'pop'] = pop_info[mask]['TPOPSRV']

In [ ]:
#mask = pop_info['pop'].isna()
#pop_info.loc[mask, 'pop'] = pop_info[mask]['tract_pop']
#np.any(pop_info['pop'].isna())

In [ ]:
#dataset = dataset[dataset['Ecode_num']==0]

In [ ]:
pop_df = pop_info[['sys_id', 'pop', 'Year']]
'sys_id' in dataset.columns
dataset = dataset.merge(pop_df, right_on=['sys_id', 'Year'], left_on=['sys_id', 'Year'] , how = 'left')

In [ ]:
dataset

In [ ]:
# categorical transformation
categorical_features = ['HUC2', 'state_id',  'KG_climate_zone' ] #, 'county_id'
ohc1 = MultiOneHotEncoder(catfeatures = categorical_features )
dataset = ohc1.transform(dataset)

In [ ]:
dataset

In [ ]:
del(dataset['population'])
del(dataset['sys_id'])

In [ ]:
dataset['wu_rate']

In [ ]:
#del(dataset['pop'])
dataset = dataset[dataset['pop']>100]
dataset['pc'] = np.log10(dataset['wu_rate']/dataset['pop'])
#del(dataset['wu_rate'])
if True:
    dataset['pop'] = np.log10(dataset['pop'] )
    dataset['wu_rate'] = np.log10(dataset['wu_rate'])
  

In [ ]:
10**dataset['pc'].quantile(0.05)

In [ ]:
#del(dataset['wu_rate'])

In [ ]:
dataset = dataset[dataset['pc']<dataset['pc'].quantile(0.98)]

In [ ]:
dataset = dataset[dataset['pc']>dataset['pc'].quantile(0.05)]

In [ ]:
del(dataset['pc'])

In [ ]:
dataset

In [ ]:
data = dataset.sample(frac=0.8, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
exp_reg101 = setup(data = data, target = 'wu_rate', train_size = 0.7, 
                   #categorical_features =categorical_features,
                   fold_shuffle= True, data_split_shuffle=True, session_id=123,
                   #normalize = True, transformation = True, 
                   #combine_rare_levels = True, rare_level_threshold = 0.05,
                   #remove_outliers = True, outliers_threshold = 0.05,
                   #pca = True, pca_method = 'kernel', pca_components = 50,
                   #create_clusters = True, cluster_iter = 20
                   #polynomial_features = True, polynomial_degree = 2,
                   #feature_interaction = True, feature_ratio = True,
                   #use_gpu = True
                  )

In [ ]:
#!jupyter nbconvert --to script 1_model_development.ipynb

In [ ]:
set_config('n_jobs_param', 2)
get_config('n_jobs_param')

In [ ]:
#best = compare_models()

In [ ]:
data

In [ ]:
xgb1 = create_model('xgboost', fold = 3)

In [ ]:
#boosted_dt = ensemble_model(xgb1, method = 'Boosting')

In [ ]:
#tuned_xgb1 = tune_model(xgb1, search_algorithm = 'random') #, n_iter = 100
# use obj func 

In [ ]:
plot_model(xgb1)

In [ ]:
plot_model(xgb1, plot = 'error')

In [ ]:
#plot_model(xgb1, plot='feature')

In [ ]:
xgb1.

In [ ]:
unseen_predictions = predict_model(xgb1, data=data_unseen)
unseen_predictions
r2 = check_metric(unseen_predictions.wu_rate/unseen_predictions['pop'], unseen_predictions.Label/unseen_predictions['pop'], 'R2')
plt.scatter(10**(unseen_predictions.wu_rate/unseen_predictions['pop']), 10**(unseen_predictions.Label/unseen_predictions['pop']))
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.plot([20, 300], [20,300], 'r')
plt.title("Unseen Data -- $R^2$ = {}".format(r2 ))

# Predicting Per capita water use directly 

In [ ]:
## predict PC
dataset['pc'] = (10**dataset['wu_rate'])/(10**dataset['pop'])
dataset['pcq'] =  
del(dataset['wu_rate'])

In [ ]:
data2 = dataset.sample(frac=0.8, random_state=786)
data_unseen2 = dataset.drop(data2.index)

data2.reset_index(drop=True, inplace=True)
data_unseen2.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
exp_reg101 = setup(data = data2, target = 'pc', train_size = 0.7, 
                   #categorical_features =categorical_features,
                   fold_shuffle= True, data_split_shuffle=True, session_id=123,
                   #normalize = True, transformation = True, 
                   #combine_rare_levels = True, rare_level_threshold = 0.05,
                   #remove_outliers = True, outliers_threshold = 0.05,
                   #pca = True, pca_method = 'kernel', pca_components = 50,
                   #create_clusters = True, cluster_iter = 20
                   #polynomial_features = True, polynomial_degree = 2,
                   #feature_interaction = True, feature_ratio = True,
                   #use_gpu = True
                  )

In [ ]:
xgb2 = create_model('xgboost', fold = 3)

In [ ]:
plot_model(xgb2)


In [ ]:
plot_model(xgb2, plot = 'error')

In [ ]:
unseen_predictions2 = predict_model(xgb2, data=data_unseen2)

r2 = check_metric(unseen_predictions2['pc'], unseen_predictions2.Label, 'R2')
plt.scatter((unseen_predictions2['pc']/unseen_predictions2['pop']), (unseen_predictions2.Label/unseen_predictions2['pop']))
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.plot([20, 300], [20,300], 'r')
plt.title("Unseen Data -- $R^2$ = {}".format(r2 ))

In [ ]:
## predict PC
dataset['pc'] =np.log10(dataset['pc'])

data3 = dataset.sample(frac=0.8, random_state=786)
data_unseen3 = dataset.drop(data3.index)

data3.reset_index(drop=True, inplace=True)
data_unseen3.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data3.shape))
print('Unseen Data For Predictions: ' + str(data_unseen3.shape))
exp_reg101 = setup(data = data3, target = 'pc', train_size = 0.7, 
                   #categorical_features =categorical_features,
                   fold_shuffle= True, data_split_shuffle=True, session_id=123,
                   #normalize = True, transformation = True, 
                   #combine_rare_levels = True, rare_level_threshold = 0.05,
                   #remove_outliers = True, outliers_threshold = 0.05,
                   #pca = True, pca_method = 'kernel', pca_components = 50,
                   #create_clusters = True, cluster_iter = 20
                   #polynomial_features = True, polynomial_degree = 2,
                   #feature_interaction = True, feature_ratio = True,
                   #use_gpu = True
                  )
xgb3 = create_model('xgboost', fold = 3)

In [ ]:
plot_model(xgb3, plot = 'error')

In [ ]:
unseen_predictions3 = predict_model(xgb3, data=data_unseen3)
unseen_predictions3
r2 = check_metric(unseen_predictions3['pc'], unseen_predictions3.Label, 'R2')
plt.scatter(10**10**(unseen_predictions3['pc']), 10**10**(unseen_predictions3.Label))
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.plot([20, 600], [20,600], 'r')
plt.title("Unseen Data -- $R^2$ = {}".format(r2 ))